In [2]:
import pandas as pd
from sqlalchemy import create_engine
import logging
import sys
DATABASE_CONFIG = {
    'host': 'localhost',
    'port': 3310,
    'user': 'root',
    'password': 'root',
    'database': 'retail_db'
}

CSV_FILES = {
    'customers': '../data/customers.csv',
    'departments': '../data/departments.csv',
    'categories': '../data/categories.csv',
    'products': '../data/products.csv',
    'orders': '../data/orders.csv',
    'order_items': '../data/order_items.csv',
}

LOG_FILE = 'logs/pipeline.log'

In [1]:
def create_db_engine(DATABASE_CONFIG):
    """
    Crear una conexion .....
    """

    try:
        engine = create_engine(
            f"mysql+mysqlconnector://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
        )
        return engine
    except:
        print("Error en la conexion")


In [3]:
def read_csv(file_path):
    """ """
    try:
        df = pd.read_csv(file_path)
        return df
    except:
        print("Error al leer el archivo")

In [4]:
df_customers = read_csv(CSV_FILES['customers'])

In [19]:
df_categories = read_csv(CSV_FILES['categories'])


In [6]:
df_departments= read_csv(CSV_FILES['departments'])
df_orders= read_csv(CSV_FILES['orders'])
df_products= read_csv(CSV_FILES['products'])
df_order_items= read_csv(CSV_FILES['order_items'])


In [7]:
def transform_customers(df):
    """
    """

    df['customer_email'] = df['customer_email'].str.lower()

    if df[['customer_fname', 'customer_lname', 'customer_email']].isnull().any().any():
        print("Datos faltantes")
        sys.exit(1)
    
    return df

In [8]:
def transform_departments(df):
    
    if df['department_name'].duplicated().any():
        print("Hay departamentos duplicados")
        sys.exit(1)
    return df
    

In [9]:
def transform_categories(df, df_departments):
    """
    """
    valid_ids = set(df_departments['department_id'])

    if not df['category_department_id'].isin(valid_ids).all():
        print("hay category_department_id que no se encuentran en departments")
        sys.exit(1)
    return df

In [10]:
def transform_poducts(df, df_categories):
    """
    """
    valid_ids = set(df_categories['category_id'])

    if not df['product_category_id'].isin(valid_ids).all():
        print("hay category_department_id que no se encuentran en departments")
        sys.exit(1)
    return df

In [21]:
def tarnsform_orders(df, df_customers):

    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
    if df['order_date'].isnull().any():
        print("Hay valores nulos")
    
    valid_ids = set(df_customers['customer_id'])
    validate_ids(df, valid_ids, 'order_customer_id')
    
    return df

In [12]:
def transform_order_item(df, df_orders, df_products):
    """
    """

    calcular_subtotal = df['order_item_quantity'] * df['order_item_product_price']
    if not (df['order_item_subtotal'] == calcular_subtotal).all():
        print("Recalculando order_item_subtotal")
        df['order_item_subtotal'] = calcular_subtotal
    
    return df

In [13]:
df_customers.head()

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,725
3,4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
4,5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,10 Crystal River Mall,Caguas,PR,725


In [14]:
def load_data(engine, table_name, df):
    df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    print("Carga exitosa")

In [15]:
engine = create_db_engine(DATABASE_CONFIG)
table_name = 'customers'

load_data(engine, table_name, df_customers)

Carga exitosa


In [17]:
def transform_categories(df, df_departments):
    """
    """
    valid_ids = set(df_departments['department_id'])

    validate_ids(df, valid_ids, 'category_department_id' )
    return df

def validate_ids(df, valid_ids, col):
    """
    """
    if not df[col].isin(valid_ids).all():
        print(f"hay {col} que no se encuentran en el otro dataframe")
        sys.exit(1)




In [20]:
transform_categories(df_categories, df_departments)

hay category_department_id que no se encuentran en el otro dataframe


SystemExit: 1

/home/codespace/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [22]:
tarnsform_orders(df_orders, df_customers)

hay order_customer_id que no se encuentran en el otro dataframe


SystemExit: 1

/home/codespace/.local/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
